MS peak align based on bioAssay peak alignment for excipient screening
see nb-dejejord-5094579-001

In [1]:
#Loading Modules
import pandas as pd #handling of csv files and dataframes
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import streamlit as st


In [2]:
st.set_page_config(page_title="Peak Alignment App", layout="wide")
st.title("LC Peak Alignment Tool")
st.markdown(
    "Hackathon Team Members: Kabir Dhingra, David Gray, Stephanie Mozley, Andrew Sinegra, and Kevin Wang "
)
st.markdown("---")

2024-09-06 13:53:43.242 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:43.246 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:44.149 
  command:

    streamlit run C:\Users\sinegraa\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-06 13:53:44.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:44.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:44.152 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:44.153 Thread 'MainThread': missing ScriptRunContext! This warning ca

DeltaGenerator()

In [3]:
# File uploader widget
uploaded_file = st.file_uploader("Upload an excel file of the peaks", type=["xlsx"])

# Check if a file has been uploaded
if uploaded_file is not None:
    # Read the uploaded file as a DataFrame
    data = pd.read_excel(uploaded_file)
    
    # Display the uploaded DataFrame
    st.write("Uploaded DataFrame:")
    st.write(data)

2024-09-06 13:53:44.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:44.190 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:44.194 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:53:44.198 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


pre-processing and qc steps
return parent peak retention time median, min RT, max RT, enter global parent peak RT (optional)
parent API peak cutoff


In [5]:
#save ./exampledata/Week2_Trimmed.xlsx as data
data = pd.read_excel("./exampledata/Week2_Trimmed.xlsx")

In [6]:
df_vial_counts = data.groupby(['Vial']).size().reset_index(name='peakCount')
maxarea = data.groupby(['Vial'])['Area'].max().reset_index()
areasum = data.groupby(['Vial'])['Area'].sum().reset_index(name='Sum of Area')
maxarea['Area Sum'] = areasum['Sum of Area']
maxarea['Area Ratio'] = maxarea['Area']/ maxarea['Area Sum']*100
RTmax= (data.sort_values(['Vial', 'Area'], ascending=[True, False])
             .drop_duplicates(['Vial']).reset_index(drop=True)
          )
maxarea = pd.merge(maxarea,RTmax,how = 'left', on = ['Vial', 'Area'])
maxarea = pd.merge(maxarea,df_vial_counts, how = 'left', on = 'Vial')
maxarea[['Plate','Vial_Well']] = maxarea['Vial'].str.split(':',expand=True)

In [7]:

median_retention_time = maxarea['Retention Time'].median()
min_retention_time = maxarea['Retention Time'].min()
max_retention_time = maxarea['Retention Time'].max()
#print median, min and max retention time
print('Median retention time:', median_retention_time)
print('Minimum retention time:', min_retention_time)
print('Maximum retention time:', max_retention_time)

Median retention time: 8.903
Minimum retention time: 8.852
Maximum retention time: 9.987


calculate RRT values, count of vial peaks, sum of AreaRatio, Assign noise, Shift global RRT option to merge

In [8]:
#calculate relative rention time 'RRT' of all of the peaks
data['RRT'] = data.apply(lambda row: row['Retention Time'] / maxarea[maxarea['Vial'] == row['Vial']]['Retention Time'].values[0], axis=1)
data['RRT'] = data['RRT'].round(2)
#calculate Area ratio of all of the peaks
data['AreaRatio'] = data.apply(lambda row: row['Area']/ maxarea[maxarea['Vial'] == row['Vial']]['Area'].values[0], axis=1)
#count number of rows with each RRT value
df_RRT_counts = data.groupby(['RRT']).size().reset_index(name='peakCount')
#find the RRT value with the highest number of peaks
max_RRT = df_RRT_counts[df_RRT_counts['peakCount'] == df_RRT_counts['peakCount'].max()]['RRT'].values[0]
#return the sum of the area ratio values for each RRT value
df_RRT_area_ratio = data.groupby(['RRT'])['AreaRatio'].sum().reset_index(name='AreaRatio Sum')
df_RRT = pd.merge(df_RRT_counts, df_RRT_area_ratio, on='RRT')
print(df_RRT.head(10))

    RRT  peakCount  AreaRatio Sum
0  0.11          1       0.001787
1  0.32          4       0.037787
2  0.33          1       0.048430
3  0.34          1       0.005200
4  0.90          1       0.000000
5  1.00        145     144.000000
6  1.12        138       0.532429
7  1.17          7       0.622475
8  1.18          9       1.920207
9  1.41          1       0.002448


In [9]:
#subset data to only the rows SampleName, Vial, RRT, and AreaRatio
data = data[['SampleName', 'Vial', 'RRT', 'AreaRatio']]

After you choose which peaks to shift, show what you are shifting in a table essentially

In [10]:
selected_rows_df = pd.DataFrame(columns=data.columns)
df_RRT['shift global RRT'] = 'None'
# Streamlit title
st.title("Select global RRT values for merging peaks")

# Function to find the row with the next least 'RRT' value for the same 'vial'
def find_next_least_rrt(df, vial, rrt):
    subset_df = df[(df['Vial'] == vial) & (df['RRT'] > rrt)]
    if not subset_df.empty:
        next_row = subset_df.loc[subset_df['RRT'].idxmin()]
        return next_row
    else:
        return None

# Function to find the row with the next highest 'RRT' value for the same 'vial'
def find_next_highest_rrt(df, vial, rrt):
    subset_df = df[(df['Vial'] == vial) & (df['RRT'] < rrt)]
    if not subset_df.empty:
        next_row = subset_df.loc[subset_df['RRT'].idxmax()]
        return next_row
    else:
        return None

# Display dropdown menus for each row
st.write("Select an action for each row:")
for index, row in df_RRT.iterrows():
    # Dropdown for selecting an action with 'None' as the default option
    shiftaction = st.selectbox(
        f"Action for {row['shift global RRT']}:",
        options=['None', '< shift to left', '> shift to right'],
        index=0,  # 'None' is the default option
        key=index  # Use the row index as the key for unique identification
    )

    # if 'None' is selected, add the rows from each vial with this RRT value to the 'selected_rows_df'
    if shiftaction == 'None':
        selected_rows_df = selected_rows_df.append(data[data['RRT'] == row['RRT']], ignore_index=True)

    
    # If "< shift to left" is selected, find the next lowest RRT row and combine their arearatios and add to 'selected_rows_df
    if shiftaction == '< shift to left':
        next_row = find_next_least_rrt(data, row['Vial'], row['RRT'])
        if next_row is not None:
            next_row['SampleName'] = row['SampleName']
            next_row['Vial'] = row['Vial']
            next_row['RRT'] = next_row['RRT']
            next_row['AreaRatio'] = row['AreaRatio'] + next_row['AreaRatio']
            selected_rows_df = selected_rows_df.append(next_row, ignore_index=True)
    
    # If "> shift to right" is selected, find the next highest RRT row and combine their arearatios and add to `selected_rows_df`
    elif shiftaction == '> shift to right':
        next_row = find_next_highest_rrt(data, row['Vial'], row['RRT'])
        if next_row is not None:
            next_row['SampleName'] = row['SampleName']
            next_row['Vial'] = row['Vial']
            next_row['RRT'] = next_row['RRT']
            next_row['AreaRatio'] = row['AreaRatio'] + next_row['AreaRatio']
            selected_rows_df = selected_rows_df.append(next_row, ignore_index=True)
# Display the updated DataFrame
pivoted_df = selected_rows_df.pivot_table(index=['SampleName', 'Vial'], columns='RRT', values='AreaRatio', fill_value=0)
st.write("Selected Rows DataFrame:")
st.write(selected_rows_df)
# Convert the DataFrame to a CSV format
csv = pivoted_df.to_csv(index=False)
st.download_button(
    label="Download Output as CSV",
    data=csv,
    file_name='selected_rows.csv',
    mime='text/csv',
)


2024-09-06 13:56:43.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:56:43.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:56:43.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:56:43.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:56:43.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:56:43.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:56:43.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 13:56:43.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

False

In [11]:
print(pivoted_df.head(10))

RRT                0.11  0.32  0.33  0.34  1.00      1.12      1.17  1.18  \
SampleName Vial                                                             
2Week_40C  1:A,1    0.0   0.0   0.0   0.0     1  0.003833  0.000000   0.0   
           1:A,10   0.0   0.0   0.0   0.0     1  0.003898  0.000000   0.0   
           1:A,11   0.0   0.0   0.0   0.0     1  0.003819  0.000000   0.0   
           1:A,12   0.0   0.0   0.0   0.0     1  0.003856  0.000000   0.0   
           1:A,2    0.0   0.0   0.0   0.0     1  0.003874  0.000000   0.0   
           1:A,3    0.0   0.0   0.0   0.0     1  0.003795  0.000000   0.0   
           1:A,4    0.0   0.0   0.0   0.0     1  0.003948  0.000000   0.0   
           1:A,5    0.0   0.0   0.0   0.0     1  0.003921  0.006186   0.0   
           1:A,6    0.0   0.0   0.0   0.0     1  0.003899  0.016116   0.0   
           1:A,7    0.0   0.0   0.0   0.0     1  0.003888  0.081484   0.0   

RRT                1.41      1.42  1.53  
SampleName Vial                  